# OpenAlex Local - Quickstart Guide

This notebook demonstrates the core features of `openalex-local`:

1. **Search** - Full-text search across 459M+ scholarly works
2. **Get** - Retrieve works by DOI or OpenAlex ID
3. **Citations** - Generate APA and BibTeX citations
4. **Cache** - Local caching for offline analysis
5. **Async** - Concurrent search operations

In [ ]:
# Import openalex-local
import openalex_local as oal
from openalex_local import search, get, cache, aio, enrich_ids

## 1. Basic Search

Search across titles and abstracts using FTS5 syntax.

In [ ]:
# Simple search
results = search("machine learning neural networks", limit=5)

print(f"Found {results.total:,} matches in {results.elapsed_ms:.1f}ms\n")

for i, work in enumerate(results.works, 1):
    print(f"{i}. {work.title} ({work.year})")
    print(f"   DOI: {work.doi or 'N/A'}")
    print()

## 2. Get by DOI

Retrieve a specific work with full metadata.

In [ ]:
# Get work by DOI
work = get("10.7717/peerj.4375")

if work:
    print(f"Title: {work.title}")
    print(f"Year: {work.year}")
    print(f"Citations: {work.cited_by_count:,}")
    print(f"Authors: {', '.join(work.authors[:3])}...")
    print(f"\nAbstract preview: {work.abstract[:200]}...")

## 3. Citations

Generate formatted citations in APA or BibTeX style.

In [ ]:
# APA citation
print("APA Citation:")
print(work.citation("apa"))
print()

# BibTeX entry
print("BibTeX Entry:")
print(work.citation("bibtex"))

## 4. Cache Workflow

Create local caches for offline analysis.

In [ ]:
# Create a cache from search
if cache.exists("demo_cache"):
    cache.delete("demo_cache")

info = cache.create("demo_cache", query="CRISPR", limit=50)
print(f"Created cache with {info.count} papers")

# Get statistics
stats = cache.stats("demo_cache")
print(f"Year range: {stats['year_min']} - {stats['year_max']}")
print(f"Mean citations: {stats['citations_mean']:.1f}")

# Query with filters
recent = cache.query("demo_cache", year_min=2020, limit=5)
print(f"\nRecent papers (2020+): {len(recent)}")

# Cleanup
cache.delete("demo_cache")
print("Cache deleted")

## 5. Async Search

Run concurrent searches for better performance.

In [ ]:
import asyncio

async def concurrent_search():
    queries = ["machine learning", "deep learning", "neural networks"]
    
    # Run all searches concurrently
    counts = await aio.count_many(queries)
    
    for query, count in counts.items():
        print(f"'{query}': {count:,} matches")

# Run async function
await concurrent_search()

## 6. Enrich IDs

Fetch full metadata for a list of IDs.

In [ ]:
# Enrich a list of IDs
ids = ["W2741809807", "10.1038/nature14539"]
works = enrich_ids(ids)

for w in works:
    print(f"Title: {w.title}")
    print(f"Year: {w.year}, Citations: {w.cited_by_count:,}")
    print()

## CLI Commands

You can also use the CLI:

```bash
# Search
openalex-local search "machine learning" -n 5

# Get by DOI with citation
openalex-local search-by-doi "10.7717/peerj.4375" --citation
openalex-local search-by-doi "10.7717/peerj.4375" --bibtex

# Cache commands
openalex-local cache create mypapers -q "CRISPR" -l 100
openalex-local cache stats mypapers
openalex-local cache export mypapers refs.bib -f bibtex
openalex-local cache delete mypapers --yes
```